In [65]:
import pandas as pd

In [66]:
data = pd.read_excel('Purchase_Ledger_Entry2.xlsx',engine='openpyxl')

In [67]:
trial_balance = pd.pivot_table(data, index=['J4 - Ledger_Name'], 
                               values=['J6 - Dr Amount', 'J7 - Cr Amount'], 
                               aggfunc='sum').fillna(0)

trial_balance['Balance'] = trial_balance['J6 - Dr Amount'] - trial_balance['J7 - Cr Amount']


In [68]:
# trial_balance.to_excel('pv.xlsx')

In [69]:
# trial_balance

In [70]:
print("sum of debit", trial_balance['J6 - Dr Amount'].sum())
print("sum of credit", trial_balance['J7 - Cr Amount'].sum())

sum of debit 7851875
sum of credit 7501600


In [71]:
data2 = pd.pivot_table(data, index=['J4 - Ledger_Name','J2 - Date'], 
                               values=['J6 - Dr Amount', 'J7 - Cr Amount'], 
                               aggfunc='sum').fillna(0)
data2['Balance'] = (data2['J6 - Dr Amount'] - data2['J7 - Cr Amount'])

account_summary = data2.groupby(['J4 - Ledger_Name','J2 - Date']).agg({'J6 - Dr Amount': 'sum', 'J7 - Cr Amount': 'sum', 'Balance': 'sum'})

# account_summary = account_summary.sort_values(by='Balance', ascending=False)

# Display the summary
print("Account Summary:")
account_summary

Account Summary:


J6 - Dr Amount  J7 - Cr Amount  Balance
J4 - Ledger_Name      J2 - Date                                          
AC                    2024-01-23           80000               0    80000
Accounts Payable      2024-02-10          100000               0   100000
Advertising Revenue   2024-02-18          100000               0   100000
Amazon                2024-01-15           44800           44800        0
Angel Investors Group 2024-02-06          360000          360000        0
...                                          ...             ...      ...
tax igst 28%          2024-01-19            7000               0     7000
                      2024-01-23           11200               0    11200
tax igst 5%           2024-01-17            1125               0     1125
                      2024-02-10            5000               0     5000
tax igst 8%           2024-02-08           16000               0    16000

[90 rows x 3 columns]

In [72]:
dr = trial_balance.groupby('J4 - Ledger_Name')['J6 - Dr Amount'].sum()
ledger = list(dr.keys())
amount = list(dr.values)
mapp_dr = {}
for i in range(len(ledger)):
    mapp_dr[ledger[i].lower()] = amount[i]

In [73]:
# dr

In [74]:
cr = trial_balance.groupby('J4 - Ledger_Name')['J7 - Cr Amount'].sum()
ledger = list(cr.keys())
amount = list(cr.values)
mapp_cr = {}
for i in range(len(ledger)):
    mapp_cr[ledger[i].lower()] = amount[i]

In [75]:
# cr

Ledger Account wise sum

In [76]:
bl = trial_balance.groupby('J4 - Ledger_Name')['Balance'].sum()
ledger = list(bl.keys())
amount = list(bl.values)
mapp_bl = {}
for i in range(len(ledger)):
    mapp_bl[ledger[i].lower()] = amount[i]

In [77]:
# mapp_bl

In [78]:
df = pd.read_excel('main_df_p.xlsx',engine='openpyxl')
df = df[df['C5'].notnull()]

In [79]:
df['Total Balance'] = df['C5'].map(mapp_bl)
# df['Debit Sum'] = df['C5'].map(mapp_dr)
# df['Credit Sum'] = df['C5'].map(mapp_cr)
df = df.dropna(subset=['Total Balance'])

In [90]:
from openpyxl import Workbook
from openpyxl.styles import Border,Side, Font,Alignment
import os

wb = Workbook()
ws = wb.active

asset_df = df[df['C1'] == 'ASSET']
as_list = list(asset_df['C2'].unique())

data = []
org_data = ['Construction Company'," ", 'Closing Balance']
bs_sheet = ['Balance Sheet as at '," ", 'Current Period']
Date  = ['31-03-2024'," ",'05/01/15 to 05/01/16']

data.append(org_data)
data.append(bs_sheet)
data.append(Date)
data.append(['','',''])
st = ["Asset", "Notes ", " "]
data.append(st)
note = 0
for i in range(len(as_list)):
    # data.append(["","",""])
    c2_data = []
    c2_data.append([as_list[i], ""," "])
    # c2_data.append(["", "",""])

    c2_1 = asset_df[asset_df['C2'] == as_list[i]]
    summary = c2_1.groupby('C4')['Total Balance'].sum()
    sum_value = asset_df[asset_df['C2'] == as_list[i]]['Total Balance'].sum()

    for index, value in summary.items():
        if float(value)>20000:
            note +=1
            c2_data.append([index,note, value])

        else:
            c2_data.append([index," ", value])

    c2_data.append(["Total " + as_list[i] , " ", sum_value])
    data.extend(c2_data)
    data.append(["","",""])


In [91]:
liability_df = df[df['C1'] == 'LIABILITY']
as_list = list(liability_df['C2'].unique())
data.append(['','',''])
st = ["Liability", "Notes ", " "]
data.append(st)

for i in range(len(as_list)):
    # data.append(["","",""])
    c2_data = []
    c2_data.append([as_list[i], ""," "])
    # c2_data.append(["", "",""])

    c2_1 = liability_df[liability_df['C2'] == as_list[i]]
    summary = c2_1.groupby('C4')['Total Balance'].sum()
    sum_value = liability_df[liability_df['C2'] == as_list[i]]['Total Balance'].sum()

   
    for index, value in summary.items():
        if float(value)>20000:
            note +=1
            c2_data.append([index,note, value])

        else:
            c2_data.append([index," ", value])

    c2_data.append(["Total " + as_list[i] , " ", sum_value])
    data.extend(c2_data)
    data.append(["","",""])

In [92]:
capital_df = df[df['C1'] == 'Capital']


data.append(["","",""])
c2_data = [["Capital", "Notes ", " "]]
# c2_data.append(["", "",""])

summary = capital_df.groupby('C4')['Total Balance'].sum()
sum_value = capital_df['Total Balance'].sum()

for index, value in summary.items():
    if float(value)>20000:
        note +=1
        c2_data.append([index,note, value])

    else:
        c2_data.append([index," ", value])

c2_data.append([ "Total " + "Equity" , " ", sum_value])
data.extend(c2_data)
data.append(["","",""])

In [93]:
border_style = Border(top=Side(style='thin'))
font_style = Font(color="FF0000")

for i, row in enumerate(data):
    ws.append(row)
    if 'Total' in row[0] :
        print(row)
        # ws.cell(row=i + 1, column=1).border = border_style
        # ws.cell(row=i + 1, column=2).border = border_style
        ws.cell(row=i + 1, column=3).border = border_style
        # ws.cell(row=i + 1, column=1).font = font_style
        # ws.cell(row=i + 1, column=2).font = font_style
        # ws.cell(row=i + 1, column=3).font = font_style

for row in ws.iter_rows():
        for cell in row:
            if cell.column == 1:
                cell.alignment = Alignment(horizontal='left', vertical='bottom')

            elif cell.column == 2:
                cell.alignment = Alignment(horizontal='center', vertical='bottom')
        
            elif cell.column == 3:
                cell.alignment = Alignment(horizontal='right', vertical='bottom')
        
for column in ws.columns:
    max_length = 0
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 1) * 1
    ws.column_dimensions[column[0].column_letter].width = adjusted_width

os.makedirs('Balance_Sheet',exist_ok = True)
wb.save("Balance_Sheet/balance_sheet1.xlsx")

['Total CurrentAsset', ' ', -3605800]
['Total FixedAsset', ' ', 400000]
['Total Current_Liability', ' ', 233700]
['Total Long_Term_Liability', ' ', 400000]
['Total Equity', ' ', 1770000]


In [85]:
# df